In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from difflib import SequenceMatcher
import logging
import json
from typing import Dict, List, Tuple

class CrimeCategoryMapper:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO)
        
    def read_excel_data(self, excel_file: str) -> pd.DataFrame:
        """Lit et transforme les données Excel mensuelles en données annuelles."""
        try:
            df = pd.read_excel(excel_file)
            
            # Création d'un DataFrame pour les données annuelles
            yearly_data = []
            
            # Pour chaque année 2020-2021 (période de validation)
            for year in [2020, 2021]:
                # Sélectionner les colonnes de l'année
                year_cols = [col for col in df.columns if f"_{year}_" in col]
                
                # Somme des valeurs mensuelles
                yearly_sum = df[year_cols].sum(axis=1)
                
                for idx, sum_value in yearly_sum.items():
                    yearly_data.append({
                        'annee': year,
                        'categorie_excel': idx,
                        'nombre_faits': sum_value
                    })
            
            return pd.DataFrame(yearly_data)
            
        except Exception as e:
            self.logger.error(f"Erreur lors de la lecture du fichier Excel: {str(e)}")
            raise

    def read_csv_data(self, csv_file: str) -> pd.DataFrame:
        """Lit et prépare les données CSV."""
        try:
            df = pd.read_csv(csv_file, sep=';', encoding='utf-8')
            
            # Convertir l'année en format complet
            df['annee'] = df['annee'].astype(str).apply(lambda x: 2000 + int(x) if len(x) == 2 else int(x))
            
            # Filtrer pour 2020-2021
            df = df[df['annee'].isin([2020, 2021])]
            
            # Convertir les valeurs numériques
            df['faits'] = pd.to_numeric(df['faits'], errors='coerce')
            
            return df
            
        except Exception as e:
            self.logger.error(f"Erreur lors de la lecture du fichier CSV: {str(e)}")
            raise

    def aggregate_csv_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Agrège les données CSV par année et catégorie."""
        return df.groupby(['annee', 'classe'])['faits'].sum().reset_index()

    def calculate_similarity_scores(self, excel_data: pd.DataFrame, csv_data: pd.DataFrame) -> Dict:
        """Calcule les scores de similarité entre les catégories basés sur les patterns de données."""
        similarity_scores = {}
        
        # Pivot des données pour avoir le même format
        excel_pivot = excel_data.pivot(index='categorie_excel', columns='annee', values='nombre_faits')
        csv_pivot = csv_data.pivot(index='classe', columns='annee', values='faits')
        
        # Normalisation des données
        excel_normalized = excel_pivot.div(excel_pivot.sum())
        csv_normalized = csv_pivot.div(csv_pivot.sum())
        
        for excel_cat in excel_normalized.index:
            similarity_scores[excel_cat] = {}
            excel_pattern = excel_normalized.loc[excel_cat].values
            
            for csv_cat in csv_normalized.index:
                csv_pattern = csv_normalized.loc[csv_cat].values
                
                # Calcul de la différence relative
                similarity = 1 - np.mean(np.abs(excel_pattern - csv_pattern))
                similarity_scores[excel_cat][csv_cat] = similarity
                
        return similarity_scores

    def create_mapping(self, similarity_scores: Dict) -> Dict:
        """Crée le mapping final basé sur les meilleurs scores de similarité."""
        mapping = {}
        used_csv_categories = set()
        
        # Trier les correspondances par score
        sorted_pairs = []
        for excel_cat, scores in similarity_scores.items():
            for csv_cat, score in scores.items():
                sorted_pairs.append((excel_cat, csv_cat, score))
                
        sorted_pairs.sort(key=lambda x: x[2], reverse=True)
        
        # Assigner les meilleures correspondances
        for excel_cat, csv_cat, score in sorted_pairs:
            if excel_cat not in mapping and csv_cat not in used_csv_categories:
                mapping[excel_cat] = {
                    'categorie_csv': csv_cat,
                    'score_similarite': score
                }
                used_csv_categories.add(csv_cat)
                
        return mapping

    def validate_mapping(self, mapping: Dict, excel_data: pd.DataFrame, csv_data: pd.DataFrame) -> Dict:
        """Valide le mapping en comparant les données."""
        validation_results = {}
        
        for excel_cat, map_info in mapping.items():
            csv_cat = map_info['categorie_csv']
            
            # Extraire les données pour chaque catégorie
            excel_values = excel_data[excel_data['categorie_excel'] == excel_cat]
            csv_values = csv_data[csv_data['classe'] == csv_cat]
            
            if not excel_values.empty and not csv_values.empty:
                # Calculer la différence relative moyenne
                excel_sum = excel_values['nombre_faits'].sum()
                csv_sum = csv_values['faits'].sum()
                
                if excel_sum > 0 and csv_sum > 0:
                    diff_ratio = abs(excel_sum - csv_sum) / max(excel_sum, csv_sum)
                    validation_results[excel_cat] = {
                        'categorie_csv': csv_cat,
                        'difference_relative': diff_ratio,
                        'confiance': 'Haute' if diff_ratio < 0.2 else 'Moyenne' if diff_ratio < 0.5 else 'Basse'
                    }
                    
        return validation_results

    def process_mapping(self, excel_file: str, csv_file: str) -> Dict:
        """Fonction principale pour créer le mapping."""
        try:
            # 1. Lecture des données
            self.logger.info("Lecture des fichiers...")
            excel_data = self.read_excel_data(excel_file)
            csv_data = self.read_csv_data(csv_file)
            
            # 2. Agrégation des données CSV
            csv_aggregated = self.aggregate_csv_data(csv_data)
            
            # 3. Calcul des scores de similarité
            self.logger.info("Calcul des similarités...")
            similarity_scores = self.calculate_similarity_scores(excel_data, csv_aggregated)
            
            # 4. Création du mapping
            self.logger.info("Création du mapping...")
            mapping = self.create_mapping(similarity_scores)
            
            # 5. Validation du mapping
            self.logger.info("Validation du mapping...")
            validation_results = self.validate_mapping(mapping, excel_data, csv_aggregated)
            
            # 6. Sauvegarder les résultats
            results = {
                'mapping': mapping,
                'validation': validation_results,
                'timestamp': datetime.now().isoformat(),
                'metadata': {
                    'periode_validation': '2020-2021',
                    'nombre_categories_mappees': len(mapping)
                }
            }
            
            # Sauvegarder en JSON
            with open('category_mapping_results.json', 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            
            return results
            
        except Exception as e:
            self.logger.error(f"Erreur lors du processus de mapping: {str(e)}")
            raise

def main():
    """Fonction principale"""
    mapper = CrimeCategoryMapper()
    
    # Exemple d'utilisation
    try:
        results = mapper.process_mapping(
            excel_file="Chiffres crimes et délits depuis janvier 1996 - 2022.xlsx",
            csv_file="donnee-dep-data.gouv-2016-2024.csv"
        )
        
        print("\nRésultats du mapping:")
        for excel_cat, info in results['mapping'].items():
            validation = results['validation'].get(excel_cat, {})
            print(f"\nCatégorie Excel: {excel_cat}")
            print(f"Catégorie CSV correspondante: {info['categorie_csv']}")
            print(f"Score de similarité: {info['score_similarite']:.2f}")
            if validation:
                print(f"Confiance: {validation['confiance']}")
                print(f"Différence relative: {validation['difference_relative']:.2%}")
                
    except Exception as e:
        print(f"Erreur lors de l'exécution: {str(e)}")

if __name__ == "__main__":
    main()

INFO:__main__:Lecture des fichiers...
INFO:__main__:Calcul des similarités...
INFO:__main__:Création du mapping...
INFO:__main__:Validation du mapping...



Résultats du mapping:

Catégorie Excel: 100
Catégorie CSV correspondante: Homicides
Score de similarité: 1.00
Confiance: Moyenne
Différence relative: 27.57%

Catégorie Excel: 36
Catégorie CSV correspondante: Cambriolages de logement
Score de similarité: 1.00
Confiance: Haute
Différence relative: 16.29%

Catégorie Excel: 84
Catégorie CSV correspondante: Vols avec armes
Score de similarité: 1.00
Confiance: Haute
Différence relative: 0.26%

Catégorie Excel: 67
Catégorie CSV correspondante: Vols violents sans arme
Score de similarité: 1.00
Confiance: Haute
Différence relative: 16.42%

Catégorie Excel: 40
Catégorie CSV correspondante: Trafic de stupéfiants
Score de similarité: 1.00
Confiance: Haute
Différence relative: 11.01%

Catégorie Excel: 34
Catégorie CSV correspondante: Vols d'accessoires sur véhicules
Score de similarité: 1.00
Confiance: Haute
Différence relative: 7.70%

Catégorie Excel: 42
Catégorie CSV correspondante: Vols dans les véhicules
Score de similarité: 1.00
Confiance: Ha

In [3]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple
import logging
from datetime import datetime

class CrimeDataMerger:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO)
        
        # Mapping département -> région (à compléter selon vos besoins)
        self.dept_region_mapping = {
            '01': '84', '02': '32', '03': '84', '04': '93', '05': '93',
            '06': '93', '07': '84', '08': '44', '09': '76', '10': '44'
            # ... Ajouter tous les mappings nécessaires
        }
        
        self.category_mapping = {
            '1': 'Homicides',  # Règlements de compte entre malfaiteurs
            '2': 'Homicides'   # Homicides pour voler et à l'occasion de vols
            # Ajouter les autres catégories selon le mapping
        }

    def get_demographic_data(self, dept: str, year: str) -> Tuple[str, str, str]:
        """
        Retourne les données démographiques (POP, LOG, tauxpourmille) pour un département et une année
        Pour l'instant, retourne des valeurs vides car ces données ne sont pas disponibles dans l'Excel
        """
        return ('', '', '')

    def read_csv_data(self, csv_file: str) -> pd.DataFrame:
        """Lit les données CSV avec le format exact"""
        try:
            df = pd.read_csv(csv_file, sep=';', encoding='utf-8', dtype={
                'classe': str,
                'annee': str,
                'Code.département': str,
                'Code.région': str,
                'unité.de.compte': str,
                'millPOP': str,
                'millLOG': str,
                'faits': str,
                'POP': str,
                'LOG': str,
                'tauxpourmille': str
            })
            return df
        except Exception as e:
            self.logger.error(f"Erreur lors de la lecture du fichier CSV: {str(e)}")
            raise

    def read_excel_data(self, excel_file: str) -> pd.DataFrame:
        """Lit et transforme les données Excel au format exact du CSV"""
        try:
            df = pd.read_excel(excel_file)
            transformed_data = []

            # Pour chaque année de 1996 à 2015
            for year in range(1996, 2016):
                year_suffix = str(year)[-2:]
                year_cols = [col for col in df.columns if f"_{year}_" in col]
                
                if year_cols:
                    # Pour chaque catégorie
                    for idx in df.index:
                        if str(idx) in self.category_mapping:
                            # Somme des valeurs mensuelles
                            yearly_sum = df.loc[idx, year_cols].sum()
                            
                            # Pour chaque département
                            for dept in self.dept_region_mapping:
                                transformed_data.append({
                                    'classe': self.category_mapping[str(idx)],
                                    'annee': year_suffix,
                                    'Code.département': dept,
                                    'Code.région': self.dept_region_mapping[dept],
                                    'unité.de.compte': 'victime' if self.category_mapping[str(idx)] == 'Homicides' else 'faits',
                                    'millPOP': year_suffix,
                                    'millLOG': year_suffix,
                                    'faits': str(int(yearly_sum / len(self.dept_region_mapping))),
                                    'POP': '',
                                    'LOG': '',
                                    'tauxpourmille': ''
                                })

            return pd.DataFrame(transformed_data)

        except Exception as e:
            self.logger.error(f"Erreur lors de la lecture du fichier Excel: {str(e)}")
            raise

    def merge_data(self, excel_data: pd.DataFrame, csv_data: pd.DataFrame) -> pd.DataFrame:
        """Fusionne les données en respectant le format exact"""
        try:
            # Définir l'ordre exact des colonnes
            columns = ['classe', 'annee', 'Code.département', 'Code.région', 
                      'unité.de.compte', 'millPOP', 'millLOG', 'faits', 
                      'POP', 'LOG', 'tauxpourmille']

            # S'assurer que les deux DataFrames ont le même format
            excel_data = excel_data.reindex(columns=columns)
            csv_data = csv_data.reindex(columns=columns)

            # Fusionner les données
            merged_data = pd.concat([
                excel_data[excel_data['annee'].astype(int) < 16],  # Données Excel avant 2016
                csv_data[csv_data['annee'].astype(int) >= 16]      # Données CSV à partir de 2016
            ], ignore_index=True)

            # Trier les données
            merged_data = merged_data.sort_values(['annee', 'classe', 'Code.département'])

            return merged_data

        except Exception as e:
            self.logger.error(f"Erreur lors de la fusion: {str(e)}")
            raise

    def process_merge(self, excel_file: str, csv_file: str) -> pd.DataFrame:
        """Fonction principale pour la fusion des données"""
        try:
            # 1. Lecture des fichiers
            self.logger.info("Lecture des fichiers...")
            excel_data = self.read_excel_data(excel_file)
            csv_data = self.read_csv_data(csv_file)

            # 2. Fusion des données
            self.logger.info("Fusion des données...")
            merged_data = self.merge_data(excel_data, csv_data)

            # 3. Sauvegarde du résultat
            output_file = f'crimes_delits_fusion_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            merged_data.to_csv(output_file, sep=';', index=False, encoding='utf-8')
            
            # 4. Affichage des statistiques
            print("\nExemple de données fusionnées (5 premières lignes):")
            print(merged_data.head().to_string())
            
            return merged_data

        except Exception as e:
            self.logger.error(f"Erreur lors de la fusion: {str(e)}")
            raise

def main():
    merger = CrimeDataMerger()
    try:
        merged_data = merger.process_merge(
            excel_file="Chiffres crimes et délits depuis janvier 1996  2022.xlsx",
            csv_file="donneedepdata.gouv20162024.csv"
        )
        
        print("\nVérification du format:")
        print(f"Nombre total de lignes: {len(merged_data)}")
        print(f"Colonnes présentes: {merged_data.columns.tolist()}")
        print(f"Types de données: \n{merged_data.dtypes}")
        
    except Exception as e:
        print(f"Erreur lors de l'exécution: {str(e)}")

if __name__ == "__main__":
    main()

INFO:__main__:Lecture des fichiers...
ERROR:__main__:Erreur lors de la lecture du fichier Excel: [Errno 2] No such file or directory: 'Chiffres crimes et délits depuis janvier 1996  2022.xlsx'
ERROR:__main__:Erreur lors de la fusion: [Errno 2] No such file or directory: 'Chiffres crimes et délits depuis janvier 1996  2022.xlsx'


Erreur lors de l'exécution: [Errno 2] No such file or directory: 'Chiffres crimes et délits depuis janvier 1996  2022.xlsx'
